In [20]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, random_split
from conversion import CSVToTensor

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.input_size = 9
        self.output_size = 9
        self.hidden_size = 2*27

        self.x0 = None
        self.x1 = None
        self.x2 = None
        self.x3 = None

        self.W1 = nn.Parameter(torch.randn(self.input_size, self.hidden_size))
        self.b1 = nn.Parameter(torch.randn(self.hidden_size))
        self.W2 = nn.Parameter(torch.randn(self.hidden_size, self.output_size))
        self.b2 = nn.Parameter(torch.randn(self.output_size))
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.3)
        self.crossloss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.train_data = None
        self.val_data = None
    
    def forward(self, x):
        self.x0 = x
        self.x1 = x @ self.W1 + self.b1
        self.x2 = self.relu(self.x1)
        self.x3 = self.x2 @ self.W2 + self.b2
        # x = self.dropout(x)
        return self.x3

    def load_data(self, file_path):
        dataloader = CSVToTensor(file_path)
        dataloader.create_all_tensor()
        dataset = dataloader.create_a_dataset()

        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        self.train_data, self.val_data = random_split(dataset, [train_size, val_size])
        self.train_data = DataLoader(self.train_data, batch_size=16, shuffle=True)
        self.val_data = DataLoader(self.val_data, batch_size=16, shuffle=True)

    def train_model(self, epochs):
        self.to(self.device)
        self.train()
        for epoch in range(epochs):
            epoch_loss = 0
            for src, trg in self.train_data:
                src = src.to(self.device)
                trg = trg.to(self.device)
                self.optimizer.zero_grad()
                output = self.forward(src)
                loss = self.crossloss(output, trg.argmax(dim=1))
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            avg_loss = epoch_loss / len(self.train_data)
            print(f"Epoch: {epoch}, Loss: {avg_loss:.4f}")
    
    

    

In [ ]:
# if __name__ == '__main__':
#     model = Model()
#     # choose the dataset file path
#     model.load_data('./Datasets/tic_tac_toe_500_games.csv')
#     # choose the number of epochs
#     with torch.no_grad():
#         model.W1.copy_(torch.zeros(model.input_size, model.hidden_size))
#         model.b1.copy_(torch.ones(model.hidden_size))
#         model.W2.copy_(torch.zeros(model.hidden_size, model.output_size))
#         model.b2.copy_(torch.ones(model.output_size))
#     model.train_model(11)

In [21]:
M = Model()

In [22]:
mytensor1 = torch.tensor([[1,0,1],[0,0,0],[0,0,0]], dtype=torch.float32)
mytensor1 = mytensor1.reshape(1,9)
# the output should be [0,2,0,0,0,0,0,0,0]
print(mytensor1.view(3,3))
print("-----------------")
mytensor2 = torch.tensor([[1,1,0],[0,0,0],[0,0,0]], dtype=torch.float32)
mytensor2 = mytensor2.reshape(1,9)
# the output should be [0,0,2,0,0,0,0,0,0]
print(mytensor2.view(3,3))
print("-----------------")
mytensor3 = torch.tensor([[0,1,1],[0,0,0],[0,0,0]], dtype=torch.float32)
mytensor3 = mytensor3.reshape(1,9)
# the output should be [2,0,0,0,0,0,0,0,0]
print(mytensor3.view(3,3))


tensor([[1., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])
-----------------
tensor([[1., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
-----------------
tensor([[0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [24]:
with torch.no_grad():
    M.W1.copy_(torch.zeros(M.input_size, M.hidden_size))
    M.b1.copy_(torch.zeros(M.hidden_size))
    M.W2.copy_(torch.zeros(M.hidden_size, M.output_size))
    M.b2.copy_(torch.zeros(M.output_size))

    M.W1[0, 1] = 2
    M.W1[0, 0] = -2

    M.W1[1,1] = -2
    M.W1[1, 2] = 2
    
    M.W1[2, 0] = 2
    M.W1[2, 2] = -2

    M.W2[0:M.output_size , 0:M.output_size] = torch.eye(M.output_size)
# print(M.W1)
# print(M.b1)
print(M.W2)

Parameter containing:
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0.,

In [33]:
output = M.forward(mytensor3)
x0 = M.x0
x1 = M.x1
x2 = M.x2
x3 = M.x3

print(x0)
print("----------------")
print(output)
print("----------------")

print("Output argmax :")
output.argmax()
# print(output.argmax())
# loss = mytensor1 - output
# loss.sum()

tensor([[0., 1., 1., 0., 0., 0., 0., 0., 0.]])
----------------
tensor([[2., 0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<AddBackward0>)
----------------
Output argmax :


tensor(0)

In [109]:
f1 = lambda in0, out0: out0-in0